<a href="https://colab.research.google.com/github/isabelladegen/ComputationalLogic/blob/prolexa-plus/Demo_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Demo and Testing Notebook Isabella
Notebook that makes sure Prolexa continues to work


### Installations like demo notebook


In [1]:
!apt-get install swi-prolog -qqq > /dev/null

Extracting templates from packages: 100%


### Installing Prolexa

Using --upgrade to always pull changes from the git repostiory!
*Same error as on the branch on the forked branch https://github.com/simply-logical/ComputationalLogic/*

In [2]:
!yes | pip install --upgrade git+https://github.com/isabelladegen/ComputationalLogic/ -qqq > /dev/null

  ERROR: Failed building wheel for prolexa
  DEPRECATION: prolexa was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### Functions to call Prolexa directly

These are not using the meta_grammar from Prolexa, meaning that no know proper nouns can be added


In [2]:
import os
import inspect
from pyswip import Prolog
import site; 
 
def find_prolog_path():
  packages_paths = site.getsitepackages()
  for ppath in packages_paths:
    if 'prolexa' in os.listdir(ppath):
      return os.path.join(ppath, 'prolexa/prolog')
    else:
      raise Exception(f'\'prolexa\' package was not found in site pagckages paths {packages_path}. Did you install it?')
                
pl = Prolog()

def initialise_prolexa():
  pl.consult(os.path.join(find_prolog_path(), 'prolexa.pl'))

def query(input): #also inspired from prolexa
  initialise_prolexa()
  libPrefix = 'prolexa:'
  generator = pl.query(libPrefix + handle_utterance_str(input))
  answer = list(generator)[0]['Output']
  if isinstance(answer, str):
    return answer
  return str(answer, 'utf-8')

def handle_utterance_str(text): #this is from the meta_grammar from prolexa plus!
    if text[0] != "'" and text[0] != '"' :
        text = f'"{text}"'

    text = text.replace('"', '\"')
    text = text.replace("'", '\"')

    return 'handle_utterance(1,{},Output)'.format(text)

In [3]:
query('Tell me all')

'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird'

###  Some Helper functions to make the tests easier to read

In [4]:
def test(input, expected_answer):
  print(f'Query: {input}')
  actual_answer = query(input)
  print(f'Answer: {actual_answer}\n')
  assert actual_answer == expected_answer, 'Expected Answer:  ' + expected_answer + '\n Actual answer: ' + actual_answer

def cleanup():
  query('forget everything')
  initialise_prolexa()


### Testing existing functionality

Syntax: `test(input_query, expected_answer)`

Be careful not to add new nouns to Prolexa. The `forget all` method currently deletes all the rules about that noun but not the noun itself

In [5]:
#Tell me about... queries
print("Test 'Tell me about...' queries:\n")
test('Tell me all', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird') #extened with new rules added to prolexa
test('tell me about unknownnoun', 'I heard you say,  tell me about unknownnoun , could you rephrase that please?')
test('tell me about tweety', 'I know nothing about tweety')
test('tell me about peter', 'peter is human. peter is mortal')
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')

#Is... queries
print("\nTest 'Is...' queries:\n")
test('Is unkownnoun human', 'I heard you say,  Is unkownnoun human , could you rephrase that please?')
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')
test('Is peter mortal', 'peter is mortal')
test('Is peter human', 'peter is human')
test('Who is human', 'peter is human')
test('Who is a bird', 'opus is a bird')
test('Does tweety fly', 'Sorry, I don\'t think this is the case')

#Explain... queries
print("\nTest 'Is...' queries:\n")
test('Explain why unknownnoun is mortal', 'I heard you say,  Explain why unknownnoun is mortal , could you rephrase that please?')
test('Explain why tweety is a bird','Sorry, I don\'t think this is the case')
test('Explain why peter is mortal', 'peter is human; every human is mortal; therefore peter is mortal')

#Adding new rules and forgeting rules
print("\nTest adding new rules:\n")
test('Peter is mortal', 'I already knew that Peter is mortal')

#add a rule and forget it again
test('Tweety is a bird', 'I will remember that Tweety is a bird')
test('Is tweety a bird', 'tweety is a bird')
# test('all birds tweet', 'I will remember that all birds tweet') NO longer works called directly
#test('Does tweety tweet', 'tweety tweets')
#test('Explain why tweety tweets', 'tweety is a bird; every bird is a tweet; therefore tweety is a tweet') #TODO notice the really odd transaltion <a tweet>, the comandline does not do this
test('Tell me all about tweety', 'tweety is a bird. tweety flies') 
test('Forget that tweety is a bird', 'I erased it from my memory')
test('tell me all about tweety', 'I know nothing about tweety')
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')
test('Does tweety fly', 'Sorry, I don\'t think this is the case')


Test 'Tell me about...' queries:

Query: Tell me all
Answer: every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird

Query: tell me about unknownnoun
Answer: I heard you say,  tell me about unknownnoun , could you rephrase that please?

Query: tell me about tweety
Answer: I know nothing about tweety

Query: tell me about peter
Answer: peter is human. peter is mortal

Query: Spill the beans
Answer: every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird


Test 'Is...' queries:

Query: Is unkownnoun human
Answer: I heard you say,  Is unkownnoun human , could you rephrase that please?

Query: Is tweety a bird
Answer: Sorry, I don't think this is the case

Query: Is peter mortal
Answer: peter is mortal

Query: Is peter human
Answer: peter is human

Query: Who is human
Answer: peter is human

Query: Who is a bird
Answer: opus is a bird

Q

# Testing new functionality


### Default Reasoning

In [8]:
cleanup()
#Testing new rules
test('Tell me about opus', 'opus is a bird. opus is a penguin. opus doesnt fly')
test('Tell me about peep', 'peep is a bird. peep flies')
test('Is opus a bird', 'opus is a bird')
test('Is peep a bird', 'peep is a bird')
test('Is opus a penguin', 'opus is a penguin')
test('Is peep a penguin', 'Sorry, I don\'t think this is the case')

#Tweety is not defined as a bird so
test('Explain why tweety flies', 'Sorry, I don\'t think this is the case')
cleanup()
test('Explain why peep flies', 'peep is a bird; some birds fly; therefore peep flies') 


#test('Spill the beans', 'every human is mortal. peter is human. penguins arent flys. every bird is a fly. every penguin is a bird. opus is a penguin. peep is a bird')
cleanup()


Query: Tell me about opus
Answer: opus is a bird. opus is a penguin. opus doesnt fly

Query: Tell me about peep
Answer: peep is a bird. peep flies

Query: Is opus a bird
Answer: opus is a bird

Query: Is peep a bird
Answer: peep is a bird

Query: Is opus a penguin
Answer: opus is a penguin

Query: Is peep a penguin
Answer: Sorry, I don't think this is the case

Query: Explain why tweety flies
Answer: Sorry, I don't think this is the case

Query: Explain why peep flies
Answer: peep is a bird; some birds fly; therefore peep flies



In [9]:
cleanup()
print(query('tweety flies'))
print(query('tell me all'))
cleanup()
print(query('tell me all'))


I will remember that tweety flies
every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird. tweety flies
every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird


###  Ideas for new funtionality
 Combine queryies test('Is peter human and mortal', 'peter is human')

 test('Is tweety a bird', 'tweety is a bird')
test('Birds fly', 'I will remember that birds fly')
test('Does tweety fly', 'tweety flies')


In [10]:
#Interesting error about Tweety flies
# print(query('tell me all'))
# print(query('tweety flies'))
# print(query('tell me all'))

Interestingly this problem does not exist if run directly in the backend and it correctly says:
```
prolexa> "Tweety flies".
*** utterance(Tweety flies)
*** rule([(fly(tweety):-true)])
*** answer(I will remember that Tweety flies)
I will remember that Tweety flies
*** utterance(Tell me all)
*** goal(all_rules(_77782))
*** answer(every human is mortal. peter is human. tweety flies)
every human is mortal. peter is human. tweety flies
```

In [11]:
print(query("tweety is a bird"))
print(query('tell me all'))
print(query('Tell me all about tweety'))
print(query('Does tweety fly'))
cleanup()
print(query('Does tweety fly'))
cleanup()
print(query('Explain why tweety flies'))

I will remember that tweety is a bird
every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird. tweety is a bird
tweety is a bird. tweety flies
tweety flies
Sorry, I don't think this is the case
Sorry, I don't think this is the case


In [12]:
print(query('All birds fly'))
print(query('tell me all about tweety'))
print(query('does tweety fly'))

I will remember that All birds fly
I know nothing about tweety
Sorry, I don't think this is the case


In [13]:
#Prove
#test('Prove Peter is human', '')